In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles
from scipy.spatial.transform import Rotation as R

### fucking around
from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSamples

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [16]:
name = r'round_plant2'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (1150/1187)

In [6]:
# keys = list(data.keys())
# k = keys[50]
# pcd_test = np.array(data[k]['point_cloud'])
# pcd_test = pcd_test[0:8,:]
# camera_centre = np.array(data[k]['pose'])
# camera_centre = rearrange(camera_centre, 'a -> 1 a')

In [34]:
num_points = 30
ground_truth_distance = training_data[:num_points,0]
angles = training_data[:num_points,1:3]
origins = training_data[:num_points,3:6]

In [19]:
def getSamples(origins, angles, ground_truth_distance, num_bins = 100):
    elev = angles[:,0]
    pan = angles[:,1]
    dir_x = torch.tensor(np.cos(elev)*np.cos(pan))      # [batch_size]
    dir_y = torch.tensor(np.cos(elev)*np.sin(pan))      # [batch_size]
    dir_z = torch.tensor(np.sin(elev))
    gt_tensor = torch.tensor(ground_truth_distance)

    # create a list of magnitudes with even spacing from 0 to 1
    t = torch.linspace(0,1, num_bins//2).expand(dir_x.shape[0], num_bins//2)  # [batch_size, num_bins//2]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins//2]
    
    # multiply the magnitude to ground truth distance and add 3 meter
    t = torch.sqrt(t)
    t = torch.sqrt(t)
    t2 = 2 - t
    t = torch.hstack((t, t2))       #[]
    t = rearrange(t, 'a b -> b a')  # [num_bins, batch_size]  transpose for multiplication broadcast
    t = gt_tensor*t

    # convert magnitudes into positions by multiplying it to the unit vector
    pos_x = dir_x*t     # [num_bins, batch_size]
    pos_y = dir_y*t
    pos_z = dir_z*t

    # concat them for output
    multiplied = rearrange([pos_x,pos_y,pos_z], 'c b n  -> (n b) c')   # [num_bin*batchsize, 3]
    # tile the origin values
    origins_tiled = repeat(origins, 'n c -> (n b) c', b = num_bins) # [num_bin*batch_size, 3]
    pos = torch.tensor(origins_tiled) + multiplied
    # tile the angle too
    angles_tiled = torch.tensor(repeat(angles, 'n c -> (n b) c', b = num_bins))
    return pos, angles_tiled, origins_tiled


In [35]:
sample_pos, sample_ang, sample_org = getSamples(origins, angles, ground_truth_distance, num_bins=100)

torch.Size([3000, 3])
(3000, 3)


In [36]:
# Convert the numpy array to an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(sample_pos)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])